In [ ]:
### Autonomous Navigation in CARLA: 
### Real-Time Vehicle Control with Camera Feedback

# Import necessary libraries
import carla
import time
import math
import cv2
import numpy as np
import random
import sys

# Append the CARLA Python API path (adjust this path to your CARLA installation)
sys.path.append('C:\\Carla\\PythonAPI\\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Host and port of the CARLA server
client.set_timeout(10.0)  # Timeout for connection attempts
world = client.get_world()  # Access the simulation world

# Constants
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance for throttle adjustments
MAX_STEER_DEGREES = 40  # Max steering angle in degrees
STEERING_CONVERSION = 75  # Factor to convert angle to steering input
CAMERA_POS_X = -5  # Camera's position behind the vehicle (meters)
CAMERA_POS_Z = 3   # Camera's height above the vehicle (meters)

# Font settings for OpenCV text display
font = cv2.FONT_HERSHEY_SIMPLEX
org1 = (30, 30)  # Speed display position
org2 = (30, 70)  # Steering angle display position
color = (255, 255, 255)  # Text color
fontScale = 0.5
thickness = 1

# Functions

def maintain_speed(current_speed):
    """
    Adjust throttle to maintain the preferred speed.
    """
    if current_speed >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif current_speed < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Full throttle if well below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def calculate_steering_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to the waypoint.
    """
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    return (angle_diff + 180) % 360 - 180

def get_next_waypoint(vehicle):
    """
    Retrieve the next waypoint for the vehicle.
    """
    vehicle_location = vehicle.get_transform().location
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        next_waypoints = waypoint.next(2.0)  # Next waypoint 2 meters ahead
        if next_waypoints:
            return next_waypoints[0]
    return None

def process_camera_image(image, data_dict):
    """
    Convert CARLA camera images to OpenCV-compatible format.
    """
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))[:, :, :3]  # Convert BGRA to BGR
    data_dict['image'] = array

# Main Script
try:
    # Initialize vehicle
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('mini')[0]  # Example vehicle blueprint
    spawn_points = world.get_map().get_spawn_points()
    
    if not spawn_points:
        raise RuntimeError("No spawn points available in the map!")
    
    spawn_point = spawn_points[0]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)

    # Attach an RGB camera to the vehicle
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '360')
    
    camera_transform = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))
    camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

    # Data dictionary for storing camera images
    data_dict = {'image': None}
    
    camera_sensor.listen(lambda image: process_camera_image(image, data_dict))

    # Simulation loop
    while True:
        # Get current speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = round(3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2), 1)

        # Get next waypoint and calculate steering angle
        next_waypoint = get_next_waypoint(vehicle)
        
        if next_waypoint:
            steering_angle = calculate_steering_angle(vehicle, next_waypoint)
            steering_input = max(-MAX_STEER_DEGREES, min(steering_angle, MAX_STEER_DEGREES)) / STEERING_CONVERSION

            # Adjust throttle based on current speed using maintain_speed function 
            throttle_input = maintain_speed(speed_kmh)

            # Apply control inputs to move the vehicle 
            control_input = carla.VehicleControl(throttle=throttle_input, steer=steering_input)
            vehicle.apply_control(control_input)

            # Display telemetry data on camera feed using OpenCV 
            if data_dict['image'] is not None:
                writable_image = data_dict['image'].copy()  # Create a writable copy of the image
                img_with_text1 = cv2.putText(writable_image, f'Speed: {speed_kmh:.1f} km/h', org1, font, fontScale, color, thickness)
                img_with_text2 = cv2.putText(img_with_text1, f'Steering Angle: {steering_angle:.1f}', org2, font, fontScale, color, thickness)
                cv2.imshow("Camera Feed", img_with_text2)

        else:
            print("No more waypoints available. Stopping simulation.")
            break

        # Exit simulation on 'q' key press 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Cleanup actors and resources 
    if 'vehicle' in locals() and vehicle:
        vehicle.destroy()
    
    if 'camera_sensor' in locals() and camera_sensor:
        camera_sensor.destroy()
    
    cv2.destroyAllWindows()


In [ ]:
### Autonomous Navigation in CARLA: 
### Real-Time Vehicle Control with Path Visualization

# Import necessary libraries
import carla  # CARLA simulator library
import time   # For delays
import math   # For calculations
import cv2    # For displaying camera feed (optional for debug visualization)
import numpy as np  # For handling image data
import sys    # To add CARLA API path

# Append the CARLA Python API path (update this to your CARLA installation path)
sys.path.append('Z:\CARLA_0.9.15\WindowsNoEditor\PythonAPI\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Connect to the CARLA server
client.set_timeout(10.0)  # Set a timeout for the connection

# Access the simulation world
world = client.get_world()

# Constants for controlling the vehicle
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance
MAX_STEER_DEGREES = 40  # Maximum allowed steering angle in degrees
STEERING_CONVERSION = 75  # Conversion factor for steering input

# Functions

def maintain_speed(speed_kmh):
    """
    Adjust throttle to maintain the preferred speed.

    :param speed_kmh: Current speed of the vehicle in km/h.
    :return: Throttle value (0 to 1).
    """
    if speed_kmh >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif speed_kmh < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Apply more throttle if significantly below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def get_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to a waypoint.

    :param vehicle: The vehicle actor.
    :param waypoint: The target waypoint.
    :return: Angle in degrees.
    """
    # Get vehicle and waypoint positions
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location

    # Calculate the angle
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    if angle_diff > 180:
        angle_diff -= 360
    elif angle_diff < -180:
        angle_diff += 360

    return angle_diff

def get_next_waypoint(vehicle):
    """
    Get the next waypoint for the vehicle.

    :param vehicle: The vehicle actor.
    :return: The next waypoint.
    """
    # Get the vehicle's current location
    vehicle_location = vehicle.get_transform().location

    # Find the closest waypoint
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)

    if waypoint:
        # Get the next waypoint 2 meters ahead
        next_waypoints = waypoint.next(2.0)
        if next_waypoints:
            return next_waypoints[0]
    return None

def draw_path(vehicle):
    """
    Draw the vehicle's path on the map.

    :param vehicle: The vehicle actor.
    """
    # Mark the vehicle's current position
    vehicle_location = vehicle.get_transform().location
    world.debug.draw_point(vehicle_location, size=0.2, color=carla.Color(255, 0, 0), life_time=0.1)

    # Retrieve and draw waypoints ahead
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        # Get waypoints until the end of the lane
        waypoints = waypoint.next_until_lane_end(2.0)
        for wp in waypoints:
            # Draw each waypoint
            world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(0, 255, 0), life_time=1.0)
            # Draw lines connecting the waypoints
            if wp.previous(2.0):
                for prev_wp in wp.previous(2.0):
                    world.debug.draw_line(
                        wp.transform.location, prev_wp.transform.location,
                        thickness=0.05, color=carla.Color(0, 0, 255), life_time=1.0
                    )

# Main Script
try:
    # Get the blueprint library and spawn a vehicle
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('mini')[0]  # Select a Mini car blueprint
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = spawn_points[0]  # Select the first spawn point
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)  # Spawn the vehicle

    # Main simulation loop
    while True:
        # Draw the vehicle's path on the map
        draw_path(vehicle)

        # Get the vehicle's velocity and calculate the speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)

        # Get the next waypoint
        next_waypoint = get_next_waypoint(vehicle)
        if next_waypoint:
            # Calculate the required steering angle
            angle_to_waypoint = get_angle(vehicle, next_waypoint)

            # Limit the steering input
            if angle_to_waypoint > MAX_STEER_DEGREES:
                steer = MAX_STEER_DEGREES / STEERING_CONVERSION
            elif angle_to_waypoint < -MAX_STEER_DEGREES:
                steer = -MAX_STEER_DEGREES / STEERING_CONVERSION
            else:
                steer = angle_to_waypoint / STEERING_CONVERSION

            # Adjust the throttle to maintain speed
            throttle = maintain_speed(speed_kmh)

            # Apply the control inputs to the vehicle
            control = carla.VehicleControl(throttle=throttle, steer=steer)
            vehicle.apply_control(control)
        else:
            print("No more waypoints available.")
            break

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Clean up and destroy the vehicle
    if vehicle is not None:
        vehicle.destroy()
    cv2.destroyAllWindows()


In [ ]:
### Autonomous Navigation in CARLA: 
### Real-Time Vehicle Control with cCamera Feedback and Path Visualization

# Import necessary libraries
import carla
import time
import math
import cv2
import numpy as np
import sys

# Append the CARLA Python API path (adjust this path to your CARLA installation)
sys.path.append('Z:\CARLA_0.9.15\WindowsNoEditor\PythonAPI\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Host and port of the CARLA server
client.set_timeout(10.0)  # Timeout for connection attempts
world = client.get_world()  # Access the simulation world

# Constants
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance for throttle adjustments
MAX_STEER_DEGREES = 80  # Max steering angle in degrees
STEERING_CONVERSION = 50  # Factor to convert angle to steering input
CAMERA_POS_X = -5  # Camera's position behind the vehicle (meters)
CAMERA_POS_Z = 3   # Camera's height above the vehicle (meters)

# Font settings for OpenCV text display
font = cv2.FONT_HERSHEY_SIMPLEX
org1 = (30, 30)  # Speed display position
org2 = (30, 70)  # Steering angle display position
color = (255, 255, 255)  # Text color
fontScale = 0.5
thickness = 1

# Functions
def maintain_speed(current_speed):
    """
    Adjust throttle to maintain the preferred speed.
    """
    if current_speed >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif current_speed < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Full throttle if well below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def calculate_steering_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to the waypoint.
    """
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    return (angle_diff + 180) % 360 - 180

def get_next_waypoint(vehicle):
    """
    Retrieve the next waypoint for the vehicle.
    """
    vehicle_location = vehicle.get_transform().location
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        next_waypoints = waypoint.next(2.0)  # Next waypoint 2 meters ahead
        if next_waypoints:
            return next_waypoints[0]
    return None

def process_camera_image(image, data_dict):
    """
    Convert CARLA camera images to OpenCV-compatible format.
    """
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))[:, :, :3]  # Convert BGRA to BGR
    data_dict['image'] = array

def visualize_path(vehicle):
    """
    Draw the vehicle's path on the map for visualization purposes.
    """
    waypoint = world.get_map().get_waypoint(vehicle.get_transform().location)
    if waypoint:
        waypoints = waypoint.next_until_lane_end(2.0)
        for wp in waypoints:
            world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(0, 255, 0), life_time=1.0)

# Main Script
try:
    # Initialize vehicle
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('mini')[0]  # Example vehicle blueprint
    spawn_points = world.get_map().get_spawn_points()
    if not spawn_points:
        raise RuntimeError("No spawn points available in the map!")
    spawn_point = spawn_points[0]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)

    # Attach an RGB camera
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '360')
    camera_transform = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

    # Data dictionary for storing camera images
    data_dict = {'image': None}
    camera.listen(lambda image: process_camera_image(image, data_dict))

    # Simulation loop
    while True:
        # Draw the path for visualization
        visualize_path(vehicle)

        # Get vehicle speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)

        # Get next waypoint and calculate steering angle
        next_waypoint = get_next_waypoint(vehicle)
        if next_waypoint:
            steering_angle = calculate_steering_angle(vehicle, next_waypoint)
            steering_input = max(-MAX_STEER_DEGREES, min(steering_angle, MAX_STEER_DEGREES)) / STEERING_CONVERSION

            # Adjust throttle
            throttle_input = maintain_speed(speed_kmh)

            # Apply control to the vehicle
            control = carla.VehicleControl(throttle=throttle_input, steer=steering_input)
            vehicle.apply_control(control)

            # Display telemetry data on the camera feed
            if data_dict['image'] is not None:
                img = data_dict['image'].copy()  # Create a writable copy of the image
                cv2.putText(img, f'Speed: {speed_kmh:.1f} km/h', org1, font, fontScale, color, thickness)
                cv2.putText(img, f'Steering Angle: {steering_angle:.1f}', org2, font, fontScale, color, thickness)
                cv2.imshow("Camera Feed", img)


        else:
            print("No more waypoints available. Stopping the simulation.")
            break

        # Exit simulation on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Cleanup actors and resources
    if 'vehicle' in locals() and vehicle:
        vehicle.destroy()
    if 'camera' in locals() and camera:
        camera.destroy()
    cv2.destroyAllWindows()

